In [1]:
# !pip install -U "langchain<0.2" "langchain-core<1.0.0" "langchain-huggingface<0.4" "langchain-google-genai<0.4"

In [2]:
# !pip install --upgrade "langchain==0.3.27" "langchain-core>=0.3,<0.4" "langchain-openai>=0.3,<0.4"

In [3]:
# !pip install --upgrade langchain langchain-core langchain-community

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
filepath = r"C:\Users\Dr. Bhavesh Dharmani\Documents\PyWork\PGDM_GenAI\myAPI.env"
load_dotenv(find_dotenv(filepath))   #r"C:\Users\Dr. Bhavesh Dharmani\Documents\PyWork\PGDM_GenAI\myAPI.env"))

True

In [2]:
# from langchain.chains import LLMChain, RetrievalQA
from langchain_classic.chains import LLMChain, RetrievalQA
# from langchain.prompts import PromptTemplate
from langchain_classic.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import SeleniumURLLoader

In [3]:
urls = [
    "https://www.alz.org",
    "https://www.nia.nih.gov", 
    "https://www.who.int/health-topics/dementia", 
    "https://www.cdc.gov/aging/dementia",
    "https://www.mayoclinic.org/diseases-conditions/dementia", 
    "https://www.dementia.org.au", 
    "https://www.nhs.uk/conditions/dementia", 
    "https://www.alzheimers.org.uk", 
    "https://www.dementiacarecentral.com", 
    "https://pubmed.ncbi.nlm.nih.gov",
    "https://www.hopkinsmedicine.org/health/conditions-and-diseases/dementia", 
    "https://dementiaaction.org/" 
]

In [4]:
loader = SeleniumURLLoader(urls = urls)
data = loader.load()
# print(data)

In [5]:
# Setup splitter: chunk_size in characters (adjust as needed)
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)

# `docs` is list of loaded Document objects
chunks = text_splitter.split_documents(data)
print(f"Total chunks created: {len(chunks)}")

Total chunks created: 106


In [6]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [8]:
# Build prompt template (optional, for customized queries)

template = """
You are a helpful business assistant.
Use the following context from company product data to answer the user question.

Context:
{context}

Question: {question}
Answer in Markdown (in 2 sentences maximum):
Intructions: 
1. Base your answer only on the above documents.
2. In case, the document do not have the answer say, “Sorry, I don’t know. Kindly 
contact the human assistant at ____ number. 
"""
qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template = template
)

In [17]:
# Use Gemini as the LLM
import os

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", 
                             # model_provider = "google-genai", 
                             api_key = os.environ["GEMINI_API_KEY"],
                             temperature=0.3)           # choice of tempretaure for RAG


In [18]:
# Create RetrievalQA chain (generalized)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",  # Or "map_reduce", "refine" for larger contexts
    return_source_documents=False,  # Or True, if you want doc provenance
    chain_type_kwargs={"prompt": qa_prompt}
)

In [19]:
# Run a query (identical for any supported LLM)
result = qa_chain.run("What is Dementia?")
print(result)

Dementia is an umbrella term for several diseases affecting memory, other cognitive abilities, and behavior that interfere with a person’s ability to maintain their activities of daily living. It is a major cause of disability and dependency among older people worldwide.


In [20]:
from IPython.display import Markdown, display

display(Markdown(result))

Dementia is an umbrella term for several diseases affecting memory, other cognitive abilities, and behavior that interfere with a person’s ability to maintain their activities of daily living. It is a major cause of disability and dependency among older people worldwide.

In [21]:
result = qa_chain.run("How I should take care of Dementia patients as a relative?")
display(Markdown(result))

To care for dementia patients, focus on advice about living well with dementia and explore available care and support options. You can also find more help and support through resources like Dementia UK and the Alzheimer's Society Dementia Support Forum.

In [ ]:
# prefer this to make sure pip and python point to the same env
# !pip show langchain langchain-core langchain-google-genai

In [ ]:
# import langchain, sys; print('langchain', getattr(langchain,'__version__','<no __version__>'), langchain.__file__)

In [ ]:
# import importlib, sys

In [ ]:
# try:
#     import langchain_core
#     print('langchain_core', getattr(langchain_core,'__version__','<no __version__>'), langchain_core.__file__)
# except Exception as e:
#     print('langchain_core import failed:', e)

In [ ]:

# python -c "import langchain, sys; print('langchain', getattr(langchain,'__version__','<no __version__>'), langchain.__file__)"
# python -c "import importlib, sys
# try:
#     import langchain_core
#     print('langchain_core', getattr(langchain_core,'__version__','<no __version__>'), langchain_core.__file__)
# except Exception as e:
#     print('langchain_core import failed:', e)"
